In [2]:
#Создаем соединение
import psycopg2
import getpass

conn=psycopg2.connect(user='postgres',
                     database='postgres',
                     host='localhost',
                     port='5432',
                     password=getpass.getpass('Insert password'))

Insert password········


In [38]:
# Получение данных из БД PostgreSQL, с которыми будем дальше работать, при помощи запроса SQL. 
# Запрашиваем данные из таблиц bond_description и quotes, объединенных по общему полю "ISIN".
# Нам необходимо все, что потребуется для создания матрицы платежей: размер купона, даты выплат купона, номинал, дата погашения и проч (см.результат ниже)
# При этом нужно исключить облигации, которые погашены, размещаются в данный момент и по которым случился дефолт.

import pandas as pd
query_bonds="SELECT DISTINCT bond_description.\"ISINCODE\",bond_description.\"ISSUERNAME\", bond_description.\"HAVEOFFER\", bond_description.\"COUPONTYPE\", quotes.\"BUYBACKDATE\", quotes.\"CPN\", quotes.\"CPN_DATE\", quotes.\"CPN_PERIOD\", quotes.\"FACEVALUE\", quotes.\"MAT_DATE\" FROM bond_description INNER JOIN quotes ON bond_description.\"ISINCODE\"=quotes.\"ISIN\" WHERE \"STATUS\"!='Погашен' and \"STATUS\"!='Размещается' and \"HAVEDEFAULT\"='0'"
bonds_desc=pd.read_sql_query(query_bonds,conn)

# Проверяем размерность матрицы

bonds_desc.shape

(4468, 10)

In [39]:
# Выводим первые 10 строк для проверки

bonds_desc.head(10)

,ISINCODE,ISSUERNAME,HAVEOFFER,COUPONTYPE,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE
0,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,4.99,2012-08-23,182.0,1000.0,2016-02-18
1,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,38.15,2011-02-24,182.0,1000.0,2016-02-18
2,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,38.15,2011-08-25,182.0,1000.0,2016-02-18
3,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,38.15,2012-02-23,182.0,1000.0,2016-02-18
4,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,38.15,2012-08-23,182.0,1000.0,2016-02-18
5,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,NaN,2009-08-27,NaN,1000.0,2016-02-18
6,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,NaN,2010-02-25,NaN,1000.0,2016-02-18
7,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,NaN,2010-08-26,NaN,1000.0,2016-02-18
8,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,NaN,2011-02-24,NaN,1000.0,2016-02-18
9,RU000A0GN9B5,Мосэнерго (ТГК-3),True,Переменный,2013-02-21,NaN,None,NaN,1000.0,2016-02-18


In [46]:
# Вспомогательное окно для проверки данных
bonds_desc[bonds_desc['COUPONTYPE']=='Дисконт']


,ISINCODE,ISSUERNAME,HAVEOFFER,COUPONTYPE,BUYBACKDATE,CPN,CPN_DATE,CPN_PERIOD,FACEVALUE,MAT_DATE


In [63]:
# Далее изымаем данные из dataframe по следующим видам облигаций: купонным с фиксированным доходом,
# амортизационным и облигациям с дисконтом
# Таблица по амортизационным облигациям
varied_coup=bonds_desc[['ISINCODE','CPN','CPN_DATE']][(bonds_desc['COUPONTYPE']=='Переменный')]
varied_coup.head(10)



,ISINCODE,CPN,CPN_DATE
0,RU000A0GN9B5,4.99,2012-08-23
1,RU000A0GN9B5,38.15,2011-02-24
2,RU000A0GN9B5,38.15,2011-08-25
3,RU000A0GN9B5,38.15,2012-02-23
4,RU000A0GN9B5,38.15,2012-08-23
5,RU000A0GN9B5,NaN,2009-08-27
6,RU000A0GN9B5,NaN,2010-02-25
7,RU000A0GN9B5,NaN,2010-08-26
8,RU000A0GN9B5,NaN,2011-02-24
9,RU000A0GN9B5,NaN,None


In [62]:
# Таблица по купонным облигациям с фиксированным купоном
fixed_coup=bonds_desc[['ISINCODE','CPN','CPN_DATE']][(bonds_desc['COUPONTYPE']=='Постоянный')|(bonds_desc['COUPONTYPE']=='Фиксированный')]
fixed_coup.head(10)


,ISINCODE,CPN,CPN_DATE
77,RU000A0JNYN1,34.9,2013-06-11
78,RU000A0JNYN1,34.9,2013-12-11
79,RU000A0JNYN1,34.9,2014-06-11
80,RU000A0JNYN1,34.9,2014-12-11
81,RU000A0JNYN1,34.9,2015-06-11
82,RU000A0JNYN1,34.9,2015-12-11
83,RU000A0JNYN1,35.1,2012-12-11
84,RU000A0JNYN1,35.1,2013-06-11
85,RU000A0JNYN1,35.1,2013-12-11
86,RU000A0JNYN1,35.1,2014-06-11


In [64]:
# Далее изымаем данные из dataframe по следующим видам облигаций: купонным с фиксированным доходом,
# амортизационным и облигациям с дисконтом
# Таблица по бескупонным облигациям
zero_coup=bonds_desc[['ISINCODE','CPN','CPN_DATE']][(bonds_desc['COUPONTYPE']=='Дисконт')]
zero_coup.head(10)

,ISINCODE,CPN,CPN_DATE


In [ ]:
# Фидбэк:
# 1.Соединить Python c БД по API. + 1 балл
# 2.Из Python запросить информацию, необходимую для пункта 3. +1 балл
# Недостает типов ценных бумаг. 
# 3.Написать функцию, которая строит матричные характеристики рейтинговых переходов. Аргументы функции: вид рейтинга, границы периода наблюдения. На выходе функция должна выдавать не только матрицу, но и значения аргументов. Привести иллюстрационный пример использования. 
#  Попытка не предпинята. 0 баллов.
# 4. Предусмотреть исполнение SQL запроса из п.2 внутри функции из п. 3. Запрос должен храниться в выходных данных функции. 0 баллов.
# Текущая сумма: 2 балла.